## Pytorch Training Pipeline

## Important Libaray import.

In [2]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.datasets import load_breast_cancer

## Load data, Split data, Scale data, Encode data.

In [22]:
df = load_breast_cancer()
X = df.data
y = df.target

# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2)

# Scaling
sc = StandardScaler()
X_train_new = sc.fit_transform(X_train)
X_test_new = sc.transform(X_test)

# Encoding target
lenc = LabelEncoder()
y_train_new = lenc.fit_transform(y_train)
y_test_new = lenc.transform(y_test)

## Convert Numpy to Tensor array.

In [24]:
# Numpy array to tensor 
X_train = torch.from_numpy(X_train_new)
X_test = torch.from_numpy(X_test_new)
y_train = torch.from_numpy(y_train_new)
y_test = torch.from_numpy(y_test_new)

## Define The Model

In [112]:
class MySimpleNN():
    def __init__(self, X):
        self.weights = torch.rand(X.shape[1], 1, dtype = torch.float64, requires_grad=True)
        self.bias = torch.zeros(1, dtype = torch.float64, requires_grad = True)
    
    def forward(self, X):
        z = torch.matmul(X, self.weights) + self.bias
        y_pred = torch.sigmoid(z)
        return y_pred
    
    def loss_function(self, y_pred, y):
        # Clamp predictions to avooid log(0)
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)
        
        # Calculate Loss
        loss = -(y * torch.log(y_pred) + (1 - y) * torch.log(1 - y_pred)).mean()
        return loss

## Important Parameters

In [141]:
learning_rate = 0.1
epochs = 20

## Training Pipeline 

In [144]:
# Create Model
model = MySimpleNN(X_train)

# define loop for epochs
for epoch in range(epochs):
    # forward pass
    y_pred = model.forward(X_train)
    
    # Loss calculate
    loss = model.loss_function(y_pred, y_train)
    
    # backward pass
    loss.backward()
    
    # parameters update (w, b)
    with torch.no_grad():
        model.weights -= learning_rate * model.weights.grad
        model.bias -= learning_rate * model.bias.grad

    # zero gradients
    model.weights.grad.zero_()
    model.bias.grad.zero_()
    
    # print loss in each epoch
    print(f"Epoch:- {epoch + 1}, Loss:- {loss.item()}")  # loss.item() used to return number from the tensor

Epoch:- 1, Loss:- 3.986556199337279
Epoch:- 2, Loss:- 3.823488094831951
Epoch:- 3, Loss:- 3.654933796402718
Epoch:- 4, Loss:- 3.4818411872407387
Epoch:- 5, Loss:- 3.3061165407846995
Epoch:- 6, Loss:- 3.1284724321577775
Epoch:- 7, Loss:- 2.950749385815224
Epoch:- 8, Loss:- 2.773141802112456
Epoch:- 9, Loss:- 2.593429310370499
Epoch:- 10, Loss:- 2.415622402649611
Epoch:- 11, Loss:- 2.241138152904009
Epoch:- 12, Loss:- 2.068734764926322
Epoch:- 13, Loss:- 1.9011701776316965
Epoch:- 14, Loss:- 1.7404544179509562
Epoch:- 15, Loss:- 1.58690530618504
Epoch:- 16, Loss:- 1.4430084902607017
Epoch:- 17, Loss:- 1.3135669113920376
Epoch:- 18, Loss:- 1.199727851226911
Epoch:- 19, Loss:- 1.1021638264848796
Epoch:- 20, Loss:- 1.0208829015305594


## Model Evaluation 

In [147]:
with torch.no_grad():
    y_pred = model.forward(X_train)
    y_pred = (y_pred > 0.5).float()
    accuracy = (y_pred == y_test).float().mean()
    print(f"Accuracy score:- {accuracy}")

Accuracy score:- 0.5153267979621887
